In [8]:
import torch.nn as nn
import pandas as pd
import torch
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import time
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import numpy as np
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import nltk
from collections import Counter

In [12]:
l = ["Epoch"]
for i in range(1,21):
    l.append(str(i))
train_log = pd.DataFrame(columns=l)

test_csv = dict()
test_csv_new = dict()
for test_data_version in ["A","B","C","D","E"]:
    # init test file
    test_csv[test_data_version] = pd.read_csv('../data/version'+test_data_version+'_test_s.csv', header=0)
    # create a new csv df with all the original columns
    test_csv_new[test_data_version] = pd.DataFrame(test_csv[test_data_version], columns=["id","version","batch.tweet","tweet.id", "tweet_hashed", "hate.speech", "offensive.language"])

In [13]:
for rate in range(100,101):
    for label_to_class in ["hate.speech", "offensive.language"]:
        for version in ["A","B","C", "D","E"]:

            ### import some packages
            from collections import Counter, OrderedDict
            from typing import Dict, Iterable, List, Optional
            from torchtext._torchtext import Vocab as VocabPybind
            from  torchtext.vocab import Vocab
            def vocab(ordered_dict: Dict, min_freq: int = 1, specials: Optional[List[str]] = None, special_first: bool = True) -> Vocab:
                specials = specials or []
                for token in specials:
                    ordered_dict.pop(token, None)
                tokens = []
                # Save room for special tokens
                for token, freq in ordered_dict.items():
                    if freq >= min_freq:
                        tokens.append(token)
                if special_first:
                    tokens[0:0] = specials
                else:
                    tokens.extend(specials)
                return Vocab(VocabPybind(tokens, None))
            def build_vocab_from_iterator(
                iterator: Iterable,
                min_freq: int = 1,
                specials: Optional[List[str]] = None,
                special_first: bool = True,
                max_tokens: Optional[int] = None,) -> Vocab:
                counter = Counter()
                for tokens in iterator:
                    counter.update(tokens)
                specials = specials or []
                # First sort by descending frequency, then lexicographically
                sorted_by_freq_tuples = sorted(counter.items(), key=lambda x: (-x[1], x[0]))
                if max_tokens is None:
                    ordered_dict = OrderedDict(sorted_by_freq_tuples)
                else:
                    assert len(specials) < max_tokens, "len(specials) >= max_tokens, so the vocab will be entirely special tokens."
                    ordered_dict = OrderedDict(sorted_by_freq_tuples[: max_tokens - len(specials)])
                word_vocab = vocab(ordered_dict, min_freq=min_freq, specials=specials, special_first=special_first)
                return word_vocab
            
            #### bild the model
            class LSTM(nn.Module):
                def __init__(self,vocab_size, emb_size, lstm_size, hidden_size, dropout):
                    super().__init__()
                    self.emb = nn.Embedding(vocab_size, emb_size)
                    self.lstm = nn.LSTM(emb_size, lstm_size, bidirectional=True, batch_first=True)
                    self.hidden = nn.Linear(lstm_size*2, hidden_size)
                    self.linear = nn.Linear(hidden_size, 1)
                    self.sigmoid = nn.Sigmoid()
                    self.dropout = nn.Dropout(dropout)
                def forward(self, input, lengths): # input.shape: (batch_size, texts_length)
                    emb = self.emb(input) # (batch_size, texts_length, emb_size)
                    emb = self.dropout(emb)
                    packed = pack_padded_sequence(emb, lengths, batch_first=True, enforce_sorted=False)  #PackedSequence: data (packed length, lstm_size*2)
                    lstm, _ = self.lstm(packed.float())  # (batch_size, texts_length, lstm_size*2)
                    padded, _ = pad_packed_sequence(lstm, batch_first=True)
                    output = torch.max(padded, dim=1).values # max pooling, (batch_size, lstm_size*2)
                    output = self.hidden(self.dropout(output)) # (batch_size, hidden_size)
                    output = self.linear(self.dropout(output)) # (batch_size, 1)
                    output = self.sigmoid(output)
                    return output.squeeze() # (batch_size)
            EPOCHS = 20
            BATCH_SIZE = 32
            EMB_SIZE = 512
            LSTM_SIZE = 512
            HIDDEN_SIZE = 256
            DROPOUT = 0.3
            VOCAB_SIZE=5000
            LEARNING_RATE = 5e-05
            DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            tokenizer = nltk.tokenize.TweetTokenizer()
            
            ### load train data, with samples
            train_csv_full = pd.read_csv("../data_median/train_dev_split/version"+version+label_to_class+'_train_median_train.csv',header=0)
            if rate < 100:
                split_i=round(len(train_csv_full)*rate*0.01)
                train_csv = train_csv_full[:split_i]
            else:
                train_csv=train_csv_full

                
            dev_csv = pd.read_csv("../data_median/train_dev_split/version"+version+label_to_class+'_train_median_dev.csv',header=0)
            X_train = [tokenizer.tokenize(text.lower()) for text in list(train_csv["tweet_hashed"])]
            X_dev = [tokenizer.tokenize(text.lower()) for text in list(dev_csv["tweet_hashed"])]
            y_train = list(train_csv[label_to_class])
            y_dev = list(dev_csv[label_to_class])
            train_data = list(zip(X_train, y_train))
            dev_data = list(zip(X_dev, y_dev))
            vocab = build_vocab_from_iterator(X_train, max_tokens=VOCAB_SIZE, specials=["<unk>", "<pad>"])
            vocab.set_default_index(vocab["<unk>"])  # index 0 reserved for '<unk>' as default, 1 reserved for '<pad>'
            torch.save(vocab, "lstm.vocab."+label_to_class+version)

            def collate(batch, vocab, device):
                texts, labels = zip(*batch)
                lengths = [len(text) for text in texts]
                word_ids = [[vocab[word] for word in text] for text in texts]
                texts = pad_sequence([torch.LongTensor(ids) for ids in word_ids], batch_first=True, padding_value=1)
                return texts.to(device), torch.tensor(labels).to(device), torch.LongTensor(lengths)

            train_loader = DataLoader(dataset=train_data,
                                    batch_size=BATCH_SIZE,
                                    shuffle=True,
                                    collate_fn=lambda batch: collate(batch, vocab, DEVICE))
            dev_loader = DataLoader(dataset=dev_data,
                                    batch_size=BATCH_SIZE,
                                    collate_fn=lambda batch: collate(batch, vocab, DEVICE))

            model = LSTM(VOCAB_SIZE, EMB_SIZE, LSTM_SIZE, HIDDEN_SIZE, DROPOUT).to(DEVICE)
            optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
            criterion = nn.BCELoss()

            #best_dev_acc = 0
            best_dev_f1 = 0
            best_epoch = 0
            print(f'Start training rate {rate}%, {label_to_class}, {version}...')
            start_time = time.time()
            #train_acc_list = [str(rate)+"_"+label_to_class+version+"_train"]
            #dev_acc_list = [str(rate)+"_"+label_to_class+version+"_dev"]
            train_f1_list = [str(rate)+"_"+label_to_class+version+"_train"]
            dev_f1_list = [str(rate)+"_"+label_to_class+version+"_dev"]
            train_loss_list = [str(rate)+"_"+label_to_class+version+"_train_loss"]
            dev_loss_list = [str(rate)+"_"+label_to_class+version+"_dev_loss"]
            
            for epoch in range(EPOCHS):
                # train
                train_loss = 0
                train_true_list=[]
                train_preds_list=[]
                train_preds_score_list=[]
                model.train()
                ln = 0
                for texts, labels, lengths in tqdm(train_loader):
                    model.zero_grad()
                    output = model(texts, lengths)
                    preds = torch.round(output)
                    if labels.shape == preds.shape:
                        loss = criterion(output, labels.float())
                        loss.backward()
                        optimizer.step()
                        train_loss += loss.item()
                        train_preds_list.extend(preds.tolist())
                        train_true_list.extend(labels.tolist())
                        train_preds_score_list.extend(output.tolist())
                        ln+=1
                train_loss = train_loss / ln
                #train_acc = accuracy_score(train_true_list,train_preds_list)
                train_f1 = f1_score(train_true_list,train_preds_list, average="weighted")
                #train_acc_list.append(train_acc)
                train_f1_list.append(train_f1)
                train_loss_list.append(train_loss)

                # dev
                dev_loss = 0
                dev_true_list=[]
                dev_preds_list=[]
                dev_preds_score_list=[]
                model.eval()
                with torch.no_grad():
                    for texts, labels, lengths in tqdm(dev_loader):
                        output = model(texts, lengths)
                        loss = criterion(output, labels.float())
                        preds = torch.round(output)
                        dev_loss += loss.item()
                        dev_preds_list.extend(preds.tolist())
                        dev_true_list.extend(labels.tolist())
                        dev_preds_score_list.extend(output.tolist())
                dev_loss = dev_loss / len(dev_loader)
                #dev_acc = accuracy_score(dev_true_list,dev_preds_list)
                #dev_acc_list.append(dev_acc)
                dev_f1 = f1_score(dev_true_list,dev_preds_list, average="weighted")
                dev_f1_list.append(dev_f1)
                dev_loss_list.append(dev_loss)

                #print(f'Epoch {e + 1}: train loss: {train_loss:.4f}, train acc: {train_acc:.4f}')
                #print(f'Epoch {e + 1}: dev loss: {dev_loss:.4f}, dev acc: {dev_acc:.4f}')
                print(f'Epoch {e + 1}: train loss: {train_loss:.4f}, train f1: {train_f1:.4f}')
                print(f'Epoch {e + 1}: dev loss: {dev_loss:.4f}, dev f1: {dev_f1:.4f}')
                #if dev_acc > best_dev_acc:
                if dev_f1 > best_dev_f1:
                    #best_dev_acc = dev_acc
                    best_dev_f1 = dev_f1
                    best_epoch = epoch + 1
                    torch.save(model, "lstm.model."+label_to_class+version)
                    print(f'*** Epoch {epoch + 1}: dev metric higher than best dev metric, model saved!')
                print()
            sec = time.time()-start_time
            #print(f'Training finished! Best epoch is {best_epoch}, best dev acc is {best_dev_acc:.4f}, {sec} seconds used.')
            print(f'Training finished! Best epoch is {best_epoch}, best dev f1 is {best_dev_f1:.4f}, {sec} seconds used.')

            #train_log.loc[len(train_log)]=train_acc_list
            #train_log.loc[len(train_log)]=dev_acc_list
            train_log.loc[len(train_log)]=train_f1_list
            train_log.loc[len(train_log)]=dev_f1_list
            train_log.loc[len(train_log)]=train_loss_list
            train_log.loc[len(train_log)]=dev_loss_list
    
    # test
    for test_data_version in ["A","B","C","D","E"]:
        BATCH_SIZE = 64
        DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        tokenizer = nltk.tokenize.TweetTokenizer()
        def collate_for_test(batch, vocab, device):
            texts, labels = zip(*batch)
            lengths = [len(text) for text in texts]
            word_ids = [[vocab[word] for word in text] for text in texts]
            texts = pad_sequence([torch.LongTensor(ids) for ids in word_ids], batch_first=True, padding_value=1)
            return texts.to(device), torch.LongTensor(lengths)
        # preprocess the test data
        X_test = [tokenizer.tokenize(text.lower()) for text in list(test_csv_new[test_data_version]["tweet_hashed"])]
        # choose trained model version to test
        for label_to_test in ["hate.speech", "offensive.language"]:
            for version_to_test in ["A","B","C","D","E"]:
                vocab_test = torch.load("lstm.vocab."+label_to_test+version_to_test, map_location=DEVICE)
                model_test = torch.load("lstm.model."+label_to_test+version_to_test, map_location=DEVICE)
                y_test = list(test_csv_new[test_data_version][label_to_test])
                test_data = list(zip(X_test, y_test))
                test_loader = DataLoader(dataset=test_data,
                                batch_size=BATCH_SIZE,
                                collate_fn=lambda batch: collate_for_test(batch, vocab_test, DEVICE))
                model_test.eval()
                preds_list = []
                preds_scores = []
                with torch.no_grad():
                    for texts, lengths in tqdm(test_loader):
                        output = model_test(texts, lengths)
                        preds = torch.round(output)
                        preds_list.extend(preds.tolist())
                        preds_scores.extend(output.tolist())
                preds_list = [int(i) for i in preds_list]
                test_csv_new[test_data_version][str(rate)+"_"+label_to_test+"_preds_"+version_to_test] = preds_list
                test_csv_new[test_data_version][str(rate)+"_"+label_to_test+"_preds_"+version_to_test+"_scores"]=preds_scores

Start training rate 100%, offensive.language, A...


100%|██████████| 15/15 [00:00<00:00, 407.91it/s]


Epoch 1: train loss: 0.6903, train acc: 0.5281
Epoch 1: dev loss: 0.6791, dev acc: 0.6156
*** Epoch 1: dev acc higher than best dev acc, model saved!



100%|██████████| 15/15 [00:00<00:00, 400.15it/s]


Epoch 2: train loss: 0.6754, train acc: 0.6003
Epoch 2: dev loss: 0.6604, dev acc: 0.6356
*** Epoch 2: dev acc higher than best dev acc, model saved!



...

100%|██████████| 15/15 [00:00<00:00, 402.93it/s]


Epoch 19: train loss: 0.1373, train acc: 0.9477
Epoch 19: dev loss: 0.7470, dev acc: 0.7933



100%|██████████| 15/15 [00:00<00:00, 406.76it/s]


Epoch 20: train loss: 0.1178, train acc: 0.9594
Epoch 20: dev loss: 0.8574, dev acc: 0.7822

Training finished! Best epoch is 18, best dev acc is 0.8044, 10.926488637924194 seconds used.


100%|██████████| 36/36 [00:00<00:00, 360.14it/s]


In [4]:
#train_log.to_csv("lstm_train_accs_median_1to100.csv")
train_log.to_csv("lstm_train_f1_median_1to100.csv")
for test_data_version in ["A","B","C","D","E"]:
    test_csv_new[test_data_version].to_csv("lstm_test"+test_data_version+"_median_1to100.csv")